In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
from pathlib import Path

# Import API key
from api_keys import geoapify_key

In [39]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
csv_file = './output_data/cities.csv'
city_data_df = pd.read_csv(csv_file)

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,abepura,-2.6333,140.5833,32.45,58,20,1.54,ID,1713492651
1,1,katsuren-haebaru,26.3369,127.8719,25.05,60,20,5.66,JP,1713492653
2,2,utrik,11.2278,169.8474,26.96,84,8,10.32,MH,1713492654
3,3,wailua homesteads,22.0669,-159.3780,23.24,85,100,7.72,US,1713492655
4,4,grytviken,-54.2811,-36.5092,4.24,98,100,0.74,GS,1713492656


In [40]:
%%capture --no-display

# Configure the map plot of all datapoints in the dataframe.
map_plot_all = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    tiles= "OSM",
    size = "Humidity",
    scale = 0.5,
    color = "City",    
    legend=True,
    legend_position='right',
    width=800, 
    height=600,  # Adjust width and height as needed
    alpha=0.6  # Set transparency level
)

# Display the map
map_plot_all

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

In [41]:
# Narrow down the city_data_df DataFrame to find your ideal weather condition. For example:
    # Drop any rows with null values
city_data_df_clean = city_data_df.dropna()

    # Narrow down cities to fit the criteria.
        # A max temperature lower than 27 degrees but higher than 21
        # Wind speed less than 4.5 m/s
        # Zero cloudiness

ideal_weather_df = city_data_df_clean[
    (city_data_df['Max Temp'] < 27) &
    (city_data_df['Max Temp'] > 21) &
    (city_data_df['Wind Speed'] < 4.5) &
    (city_data_df['Cloudiness'] == 0)]

# Display sample data
ideal_weather_df.head()

C:\Users\morga\AppData\Local\Temp\ipykernel_26488\1928555734.py:10: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  ideal_weather_df = city_data_df_clean[


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
233,233,tanumah,27.1000,44.1333,22.80,22,0,2.42,SA,1713492940
241,241,kingston,17.9970,-76.7936,25.65,86,0,1.45,JM,1713492949
252,252,mount pleasant,32.7941,-79.8626,22.95,82,0,0.45,US,1713492963
260,260,turabah,21.2141,41.6331,22.59,47,0,1.87,SA,1713492973
317,317,roswell,34.0232,-84.3616,24.28,74,0,3.09,US,1713493043


In [46]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
233,tanumah,SA,27.1000,44.1333,22,
241,kingston,JM,17.9970,-76.7936,86,
252,mount pleasant,US,32.7941,-79.8626,82,
260,turabah,SA,21.2141,41.6331,47,
317,roswell,US,34.0232,-84.3616,74,
334,callao,PE,-12.0667,-77.1500,77,
362,al kharijah,EG,25.4514,30.5464,32,
384,adrar,MR,20.5022,-10.0711,10,
424,puerto lempira,HN,15.2667,-83.7667,82,
459,turpan,CN,42.9333,89.1667,23,


In [73]:
# Set parameters to search for a hotel.
# Set radius to 10,000.00 m (10 000 Metres)
radius = 10000
shape = 'circle'
bias = 'proximity'

params = {
    'categories': 'accommodation.hotel',
    'filter': f'{shape}:{longitude},{latitude},{radius}',
    'limit': 1,
    'apiKey': geoapify_key,
    'bias': bias
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row['Lat']
    longitude = row['Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary.
    params["filter"] = f'{shape}:{longitude},{latitude},{radius}'
    params["bias"] =   f'{bias}:{longitude},{latitude}'
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
tanumah - nearest hotel: No hotel found
kingston - nearest hotel: Aphrodite Hotel
mount pleasant - nearest hotel: Old Village Post House
turabah - nearest hotel: No hotel found
roswell - nearest hotel: Best Western Plus Hotel
callao - nearest hotel: Casa Ronald
al kharijah - nearest hotel: Kharga Hotel
adrar - nearest hotel: No hotel found
puerto lempira - nearest hotel: Hotel Yu Baiwan
turpan - nearest hotel: Oriental
fort bragg - nearest hotel: Airborne Inn Lodging
abu road - nearest hotel: Ambaji
hiratacho - nearest hotel: ひらたメイプルホテル
sur - nearest hotel: Sur Hotel
lazaro cardenas - nearest hotel: Hotel Sol del Pacífico


,City,Country,Lat,Lng,Humidity,Hotel Name
233,tanumah,SA,27.1000,44.1333,22,No hotel found
241,kingston,JM,17.9970,-76.7936,86,Aphrodite Hotel
252,mount pleasant,US,32.7941,-79.8626,82,Old Village Post House
260,turabah,SA,21.2141,41.6331,47,No hotel found
317,roswell,US,34.0232,-84.3616,74,Best Western Plus Hotel
334,callao,PE,-12.0667,-77.1500,77,Casa Ronald
362,al kharijah,EG,25.4514,30.5464,32,Kharga Hotel
384,adrar,MR,20.5022,-10.0711,10,No hotel found
424,puerto lempira,HN,15.2667,-83.7667,82,Hotel Yu Baiwan
459,turpan,CN,42.9333,89.1667,23,Oriental


In [86]:
%%capture --no-display

# Configure the map plot.
map_plot_hotels = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    tiles= "OSM",
    size = "Humidity",
    scale = 1.0,
    color = "City",    
    legend=True,
    legend_position='right',
    width=900, 
    height=600,  # Adjust width and height as needed
    alpha=0.6,  # Set transparency level
    hover_cols=["Hotel Name",'Country']
)

# Display the map
map_plot_hotels

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)